<a href="https://colab.research.google.com/github/almond5/CAP5510_Final_Project/blob/main/qna_method/qna_clinicalbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ***Packages***

In [ ]:
%pip install transformers
%pip install torch
%pip install datasets

## ***ClinicalBERT***

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

# Load ClinicalBERT tokenizer and model from "medicalai/ClinicalBERT"
tokenizer = AutoTokenizer.from_pretrained("medicalai/ClinicalBERT")
model = AutoModelForSequenceClassification.from_pretrained("medicalai/ClinicalBERT", num_labels=3)
ds = load_dataset("qiaojin/PubMedQA", "pqa_labeled")

# Move model to the available device (cuda or cpu)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at medicalai/ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)

## ***200 Samples; 5 Epochs***

### ***Trial 1***

In [ ]:
# Preprocess the dataset
def preprocess_function(examples):
    inputs = [f"Context: {context} Question: {question}" for question, context in zip(examples['question'], examples['context'])]
    targets = examples['final_decision']

    # Expanded label mapping to handle additional cases
    label_mapping = {"yes": 0, "no": 1, "uncertain": 2, "maybe": 2}

    # Handle missing or unexpected labels gracefully
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    model_inputs["labels"] = [label_mapping.get(target.lower(), 2) for target in targets]

    return model_inputs

# Select a small subset of the dataset for demonstration
small_ds = ds['train'].select(range(200))
tokenized_ds = small_ds.map(preprocess_function, batched=True)

# Split the dataset into training, validation, and test sets
train_size = 100
val_size = 50
test_size = 50

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    tokenized_ds, [train_size, val_size, test_size]
)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-6,
    num_train_epochs=5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

# Evaluate the model
metrics = trainer.evaluate(eval_dataset=test_dataset)
print("Test set evaluation:", metrics)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-42-7b022906df66>:40: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,No log,1.030395
2,No log,1.014960
3,No log,1.007244


Test set evaluation: {'eval_loss': 0.9915612936019897, 'eval_runtime': 0.3321, 'eval_samples_per_second': 60.228, 'eval_steps_per_second': 60.228, 'epoch': 3.0}


In [ ]:
# Generate predictions
def prediction_answer(question, context):
    # Combine the question and context into one input
    combined_input = f"Context: {context} Question: {question}"

    # Tokenize the input
    inputs = tokenizer(combined_input, return_tensors="pt", max_length=512, truncation=True, padding="max_length")
    inputs = {key: val.to(device) for key, val in inputs.items()}

    # Get the model's output
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_label = torch.argmax(logits, dim=1).item()

    # Map numeric labels to "yes", "no", or "maybe"
    label_map = {0: "yes", 1: "no", 2: "maybe"}
    return label_map[predicted_label]

num_correct = 0

# Evaluate the Q&A performance
for example in test_dataset:
    question = example['question']
    # Include the context field in the prediction
    context = example['context']
    # Extract true answer from 'final_decision' column
    true_answer = example['final_decision']
    # Use model to predict the answer
    predicted_answer = prediction_answer(question, context)

    print(f"Question: {question}")
    print(f"Context: {context}")
    print(f"True Answer: {true_answer}")
    print(f"Predicted Answer: {predicted_answer}")
    print("="*80)

    # Calculate accuracy
    if true_answer == predicted_answer:
        num_correct += 1

# Print accuracy
accuracy = num_correct / len(test_dataset)
print(f"Accuracy: {accuracy}")

Question: Does either obesity or OSA severity influence the response of autotitrating CPAP machines in very obese subjects?
Context: {'contexts': ['The pressures delivered by autotitrating continuous positive airways pressure (CPAP) devices not only treat obstructive sleep apnoea (OSA) effectively but also give potentially interesting physiological information about the forces impinging on the pharynx. In earlier work from this unit, we used correlations between autoCPAP pressure and both OSA severity and obesity, to construct an algorithm to estimate the fixed CPAP pressure a patient required for subsequent clinical use. We wished to discover if these relationships could be reliably extended to a much more obese group.', "We performed a prospective cohort study in an obese population. Measurements of obesity were made, OSA severity was recorded, and the 95th centile autoCPAP pressure was recorded during 1\xa0week of autoCPAP. Spearman's rank correlation was performed between measureme

### ***Trial 2***

In [ ]:
# Preprocess the dataset
def preprocess_function(examples):
    inputs = [f"Context: {context} Question: {question}" for question, context in zip(examples['question'], examples['context'])]
    targets = examples['final_decision']

    # Expanded label mapping to handle additional cases
    label_mapping = {"yes": 0, "no": 1, "uncertain": 2, "maybe": 2}

    # Handle missing or unexpected labels gracefully
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    model_inputs["labels"] = [label_mapping.get(target.lower(), 2) for target in targets]

    return model_inputs

# Select a small subset of the dataset for demonstration
small_ds = ds['train'].select(range(200))
tokenized_ds = small_ds.map(preprocess_function, batched=True)

# Split the dataset into training, validation, and test sets
train_size = 100
val_size = 50
test_size = 50

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    tokenized_ds, [train_size, val_size, test_size]
)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-6,
    num_train_epochs=5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

# Evaluate the model
metrics = trainer.evaluate(eval_dataset=test_dataset)
print("Test set evaluation:", metrics)

In [ ]:
# Generate predictions
def prediction_answer(question, context):
    # Combine the question and context into one input
    combined_input = f"Context: {context} Question: {question}"

    # Tokenize the input
    inputs = tokenizer(combined_input, return_tensors="pt", max_length=512, truncation=True, padding="max_length")
    inputs = {key: val.to(device) for key, val in inputs.items()}

    # Get the model's output
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_label = torch.argmax(logits, dim=1).item()

    # Map numeric labels to "yes", "no", or "maybe"
    label_map = {0: "yes", 1: "no", 2: "maybe"}
    return label_map[predicted_label]

num_correct = 0

# Evaluate the Q&A performance
for example in test_dataset:
    question = example['question']
    # Include the context field in the prediction
    context = example['context']
    # Extract true answer from 'final_decision' column
    true_answer = example['final_decision']
    # Use model to predict the answer
    predicted_answer = prediction_answer(question, context)

    print(f"Question: {question}")
    print(f"Context: {context}")
    print(f"True Answer: {true_answer}")
    print(f"Predicted Answer: {predicted_answer}")
    print("="*80)

    # Calculate accuracy
    if true_answer == predicted_answer:
        num_correct += 1

# Print accuracy
accuracy = num_correct / len(test_dataset)
print(f"Accuracy: {accuracy}")

### ***Trial 3***

In [ ]:
# Preprocess the dataset
def preprocess_function(examples):
    inputs = [f"Context: {context} Question: {question}" for question, context in zip(examples['question'], examples['context'])]
    targets = examples['final_decision']

    # Expanded label mapping to handle additional cases
    label_mapping = {"yes": 0, "no": 1, "uncertain": 2, "maybe": 2}

    # Handle missing or unexpected labels gracefully
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    model_inputs["labels"] = [label_mapping.get(target.lower(), 2) for target in targets]

    return model_inputs

# Select a small subset of the dataset for demonstration
small_ds = ds['train'].select(range(200))
tokenized_ds = small_ds.map(preprocess_function, batched=True)

# Split the dataset into training, validation, and test sets
train_size = 100
val_size = 50
test_size = 50

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    tokenized_ds, [train_size, val_size, test_size]
)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-6,
    num_train_epochs=5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

# Evaluate the model
metrics = trainer.evaluate(eval_dataset=test_dataset)
print("Test set evaluation:", metrics)

In [ ]:
# Generate predictions
def prediction_answer(question, context):
    # Combine the question and context into one input
    combined_input = f"Context: {context} Question: {question}"

    # Tokenize the input
    inputs = tokenizer(combined_input, return_tensors="pt", max_length=512, truncation=True, padding="max_length")
    inputs = {key: val.to(device) for key, val in inputs.items()}

    # Get the model's output
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_label = torch.argmax(logits, dim=1).item()

    # Map numeric labels to "yes", "no", or "maybe"
    label_map = {0: "yes", 1: "no", 2: "maybe"}
    return label_map[predicted_label]

num_correct = 0

# Evaluate the Q&A performance
for example in test_dataset:
    question = example['question']
    # Include the context field in the prediction
    context = example['context']
    # Extract true answer from 'final_decision' column
    true_answer = example['final_decision']
    # Use model to predict the answer
    predicted_answer = prediction_answer(question, context)

    print(f"Question: {question}")
    print(f"Context: {context}")
    print(f"True Answer: {true_answer}")
    print(f"Predicted Answer: {predicted_answer}")
    print("="*80)

    # Calculate accuracy
    if true_answer == predicted_answer:
        num_correct += 1

# Print accuracy
accuracy = num_correct / len(test_dataset)
print(f"Accuracy: {accuracy}")

## ***300 Samples; 5 Epochs***

### ***Trial 1***

In [ ]:
# Preprocess the dataset
def preprocess_function(examples):
    inputs = [f"Context: {context} Question: {question}" for question, context in zip(examples['question'], examples['context'])]
    targets = examples['final_decision']

    # Expanded label mapping to handle additional cases
    label_mapping = {"yes": 0, "no": 1, "uncertain": 2, "maybe": 2}

    # Handle missing or unexpected labels gracefully
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    model_inputs["labels"] = [label_mapping.get(target.lower(), 2) for target in targets]

    return model_inputs

# Select a small subset of the dataset for demonstration
small_ds = ds['train'].select(range(300))
tokenized_ds = small_ds.map(preprocess_function, batched=True)

train_size = 200
val_size = 50
test_size = 50

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    tokenized_ds, [train_size, val_size, test_size]
)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-6,
    num_train_epochs=5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

# Evaluate the model
metrics = trainer.evaluate(eval_dataset=test_dataset)
print("Test set evaluation:", metrics)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-49-79e6017cb264>:39: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,No log,1.261373
2,No log,0.995275
3,1.299000,0.901444
4,1.299000,0.865073
5,1.048300,0.833709


Test set evaluation: {'eval_loss': 0.7653539776802063, 'eval_runtime': 0.8342, 'eval_samples_per_second': 59.935, 'eval_steps_per_second': 59.935, 'epoch': 5.0}


In [ ]:
# Generate predictions
def prediction_answer(question, context):
    # Combine the question and context into one input
    combined_input = f"Context: {context} Question: {question}"

    # Tokenize the input
    inputs = tokenizer(combined_input, return_tensors="pt", max_length=512, truncation=True, padding="max_length")
    inputs = {key: val.to(device) for key, val in inputs.items()}  # Move inputs to the correct device

    # Get the model's output
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_label = torch.argmax(logits, dim=1).item()

    # Map numeric labels to "yes", "no", or "maybe"
    label_map = {0: "yes", 1: "no", 2: "maybe"}
    return label_map[predicted_label]

num_correct = 0

# Evaluate the Q&A performance
for example in test_dataset:
    question = example['question']
    # Include the context field in the prediction
    context = example['context']
    # Extract true answer from 'final_decision' column
    true_answer = example['final_decision']
    # Use model to predict the answer
    predicted_answer = prediction_answer(question, context)

    print(f"Question: {question}")
    print(f"Context: {context}")
    print(f"True Answer: {true_answer}")
    print(f"Predicted Answer: {predicted_answer}")
    print("="*80)

    # Calculate accuracy
    if true_answer == predicted_answer:
        num_correct += 1

# Print accuracy
accuracy = num_correct / len(test_dataset)
print(f"Accuracy: {accuracy}")

Question: Is the use of cyanoacrylate in intestinal anastomosis a good and reliable alternative?
Context: {'contexts': ['The present study aims to compare strength, healing, and operation time of experimental intestinal anastomoses performed by polyglactin 910 (Vicryl; Ethicon, Edinburgh, United Kingdom) sutures with ethyl-2-cyanoacrylate glue (Pattex; Henkel, Dusseldorf, Germany).', "Ninety-six Sprague-Dawley rats were divided into 2 (groups E and L). Each group was further subdivided into 6 subgroups (EA1, EA2, EA3, EB1, EB2, EB3, LA1, LA2, LA3, LB1, LB2, LB3), each containing 8 rats. Intestinal anastomosis was performed by polyglactin 910 sutures in A subgroups and with ethyl-2-cyanoacrylate in B subgroups. The anastomosis was end to end in A1 and B1, side to side in A2 and B2, and end to side in A3 and B3. Time for anastomosis performance (AT) was recorded. In group E, bursting pressures and hydroxyproline levels were determined on the second postoperative day, whereas in group L, 

### ***Trial 2***

In [ ]:
# Preprocess the dataset
def preprocess_function(examples):
    inputs = [f"Context: {context} Question: {question}" for question, context in zip(examples['question'], examples['context'])]
    targets = examples['final_decision']

    # Expanded label mapping to handle additional cases
    label_mapping = {"yes": 0, "no": 1, "uncertain": 2, "maybe": 2}

    # Handle missing or unexpected labels gracefully
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    model_inputs["labels"] = [label_mapping.get(target.lower(), 2) for target in targets]

    return model_inputs

# Select a small subset of the dataset for demonstration
small_ds = ds['train'].select(range(300))
tokenized_ds = small_ds.map(preprocess_function, batched=True)

train_size = 200
val_size = 50
test_size = 50

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    tokenized_ds, [train_size, val_size, test_size]
)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-6,
    num_train_epochs=5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

# Evaluate the model
metrics = trainer.evaluate(eval_dataset=test_dataset)
print("Test set evaluation:", metrics)

In [ ]:
# Generate predictions
def prediction_answer(question, context):
    # Combine the question and context into one input
    combined_input = f"Context: {context} Question: {question}"

    # Tokenize the input
    inputs = tokenizer(combined_input, return_tensors="pt", max_length=512, truncation=True, padding="max_length")
    inputs = {key: val.to(device) for key, val in inputs.items()}  # Move inputs to the correct device

    # Get the model's output
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_label = torch.argmax(logits, dim=1).item()

    # Map numeric labels to "yes", "no", or "maybe"
    label_map = {0: "yes", 1: "no", 2: "maybe"}
    return label_map[predicted_label]

num_correct = 0

# Evaluate the Q&A performance
for example in test_dataset:
    question = example['question']
    # Include the context field in the prediction
    context = example['context']
    # Extract true answer from 'final_decision' column
    true_answer = example['final_decision']
    # Use model to predict the answer
    predicted_answer = prediction_answer(question, context)

    print(f"Question: {question}")
    print(f"Context: {context}")
    print(f"True Answer: {true_answer}")
    print(f"Predicted Answer: {predicted_answer}")
    print("="*80)

    # Calculate accuracy
    if true_answer == predicted_answer:
        num_correct += 1

# Print accuracy
accuracy = num_correct / len(test_dataset)
print(f"Accuracy: {accuracy}")

### ***Trial 3***

In [ ]:
# Preprocess the dataset
def preprocess_function(examples):
    inputs = [f"Context: {context} Question: {question}" for question, context in zip(examples['question'], examples['context'])]
    targets = examples['final_decision']

    # Expanded label mapping to handle additional cases
    label_mapping = {"yes": 0, "no": 1, "uncertain": 2, "maybe": 2}

    # Handle missing or unexpected labels gracefully
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    model_inputs["labels"] = [label_mapping.get(target.lower(), 2) for target in targets]

    return model_inputs

# Select a small subset of the dataset for demonstration
small_ds = ds['train'].select(range(300))
tokenized_ds = small_ds.map(preprocess_function, batched=True)

train_size = 200
val_size = 50
test_size = 50

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    tokenized_ds, [train_size, val_size, test_size]
)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-6,
    num_train_epochs=5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

# Evaluate the model
metrics = trainer.evaluate(eval_dataset=test_dataset)
print("Test set evaluation:", metrics)

In [ ]:
# Generate predictions
def prediction_answer(question, context):
    # Combine the question and context into one input
    combined_input = f"Context: {context} Question: {question}"

    # Tokenize the input
    inputs = tokenizer(combined_input, return_tensors="pt", max_length=512, truncation=True, padding="max_length")
    inputs = {key: val.to(device) for key, val in inputs.items()}  # Move inputs to the correct device

    # Get the model's output
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_label = torch.argmax(logits, dim=1).item()

    # Map numeric labels to "yes", "no", or "maybe"
    label_map = {0: "yes", 1: "no", 2: "maybe"}
    return label_map[predicted_label]

num_correct = 0

# Evaluate the Q&A performance
for example in test_dataset:
    question = example['question']
    # Include the context field in the prediction
    context = example['context']
    # Extract true answer from 'final_decision' column
    true_answer = example['final_decision']
    # Use model to predict the answer
    predicted_answer = prediction_answer(question, context)

    print(f"Question: {question}")
    print(f"Context: {context}")
    print(f"True Answer: {true_answer}")
    print(f"Predicted Answer: {predicted_answer}")
    print("="*80)

    # Calculate accuracy
    if true_answer == predicted_answer:
        num_correct += 1

# Print accuracy
accuracy = num_correct / len(test_dataset)
print(f"Accuracy: {accuracy}")

## ***400 Samples; 5 Epochs***

### ***Trial 1***

In [ ]:
# Preprocess the dataset
def preprocess_function(examples):
    inputs = [f"Context: {context} Question: {question}" for question, context in zip(examples['question'], examples['context'])]
    targets = examples['final_decision']

    # Expanded label mapping to handle additional cases
    label_mapping = {"yes": 0, "no": 1, "uncertain": 2, "maybe": 2}

    # Handle missing or unexpected labels gracefully
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    model_inputs["labels"] = [label_mapping.get(target.lower(), 2) for target in targets]

    return model_inputs

# Select a small subset of the dataset for demonstration
small_ds = ds['train'].select(range(400))
tokenized_ds = small_ds.map(preprocess_function, batched=True)

train_size = 300
val_size = 50
test_size = 50

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    tokenized_ds, [train_size, val_size, test_size]
)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-6,
    num_train_epochs=5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

# Evaluate the model
metrics = trainer.evaluate(eval_dataset=test_dataset)
print("Test set evaluation:", metrics)

In [ ]:
# Generate predictions
def prediction_answer(question, context):
    # Combine the question and context into one input
    combined_input = f"Context: {context} Question: {question}"

    # Tokenize the input
    inputs = tokenizer(combined_input, return_tensors="pt", max_length=512, truncation=True, padding="max_length")
    inputs = {key: val.to(device) for key, val in inputs.items()}  # Move inputs to the correct device

    # Get the model's output
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_label = torch.argmax(logits, dim=1).item()

    # Map numeric labels to "yes", "no", or "maybe"
    label_map = {0: "yes", 1: "no", 2: "maybe"}
    return label_map[predicted_label]

num_correct = 0

# Evaluate the Q&A performance
for example in test_dataset:
    question = example['question']
    # Include the context field in the prediction
    context = example['context']
    # Extract true answer from 'final_decision' column
    true_answer = example['final_decision']
    # Use model to predict the answer
    predicted_answer = prediction_answer(question, context)

    print(f"Question: {question}")
    print(f"Context: {context}")
    print(f"True Answer: {true_answer}")
    print(f"Predicted Answer: {predicted_answer}")
    print("="*80)

    # Calculate accuracy
    if true_answer == predicted_answer:
        num_correct += 1

# Print accuracy
accuracy = num_correct / len(test_dataset)
print(f"Accuracy: {accuracy}")

### ***Trial 2***

In [ ]:
# Preprocess the dataset
def preprocess_function(examples):
    inputs = [f"Context: {context} Question: {question}" for question, context in zip(examples['question'], examples['context'])]
    targets = examples['final_decision']

    # Expanded label mapping to handle additional cases
    label_mapping = {"yes": 0, "no": 1, "uncertain": 2, "maybe": 2}

    # Handle missing or unexpected labels gracefully
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    model_inputs["labels"] = [label_mapping.get(target.lower(), 2) for target in targets]

    return model_inputs

# Select a small subset of the dataset for demonstration
small_ds = ds['train'].select(range(400))
tokenized_ds = small_ds.map(preprocess_function, batched=True)

train_size = 300
val_size = 50
test_size = 50

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    tokenized_ds, [train_size, val_size, test_size]
)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-6,
    num_train_epochs=5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

# Evaluate the model
metrics = trainer.evaluate(eval_dataset=test_dataset)
print("Test set evaluation:", metrics)

In [ ]:
# Generate predictions
def prediction_answer(question, context):
    # Combine the question and context into one input
    combined_input = f"Context: {context} Question: {question}"

    # Tokenize the input
    inputs = tokenizer(combined_input, return_tensors="pt", max_length=512, truncation=True, padding="max_length")
    inputs = {key: val.to(device) for key, val in inputs.items()}  # Move inputs to the correct device

    # Get the model's output
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_label = torch.argmax(logits, dim=1).item()

    # Map numeric labels to "yes", "no", or "maybe"
    label_map = {0: "yes", 1: "no", 2: "maybe"}
    return label_map[predicted_label]

num_correct = 0

# Evaluate the Q&A performance
for example in test_dataset:
    question = example['question']
    # Include the context field in the prediction
    context = example['context']
    # Extract true answer from 'final_decision' column
    true_answer = example['final_decision']
    # Use model to predict the answer
    predicted_answer = prediction_answer(question, context)

    print(f"Question: {question}")
    print(f"Context: {context}")
    print(f"True Answer: {true_answer}")
    print(f"Predicted Answer: {predicted_answer}")
    print("="*80)

    # Calculate accuracy
    if true_answer == predicted_answer:
        num_correct += 1

# Print accuracy
accuracy = num_correct / len(test_dataset)
print(f"Accuracy: {accuracy}")

### ***Trial 3***

In [ ]:
# Preprocess the dataset
def preprocess_function(examples):
    inputs = [f"Context: {context} Question: {question}" for question, context in zip(examples['question'], examples['context'])]
    targets = examples['final_decision']

    # Expanded label mapping to handle additional cases
    label_mapping = {"yes": 0, "no": 1, "uncertain": 2, "maybe": 2}

    # Handle missing or unexpected labels gracefully
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    model_inputs["labels"] = [label_mapping.get(target.lower(), 2) for target in targets]

    return model_inputs

# Select a small subset of the dataset for demonstration
small_ds = ds['train'].select(range(400))
tokenized_ds = small_ds.map(preprocess_function, batched=True)

train_size = 300
val_size = 50
test_size = 50

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    tokenized_ds, [train_size, val_size, test_size]
)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-6,
    num_train_epochs=5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

# Evaluate the model
metrics = trainer.evaluate(eval_dataset=test_dataset)
print("Test set evaluation:", metrics)

In [ ]:
# Generate predictions
def prediction_answer(question, context):
    # Combine the question and context into one input
    combined_input = f"Context: {context} Question: {question}"

    # Tokenize the input
    inputs = tokenizer(combined_input, return_tensors="pt", max_length=512, truncation=True, padding="max_length")
    inputs = {key: val.to(device) for key, val in inputs.items()}  # Move inputs to the correct device

    # Get the model's output
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_label = torch.argmax(logits, dim=1).item()

    # Map numeric labels to "yes", "no", or "maybe"
    label_map = {0: "yes", 1: "no", 2: "maybe"}
    return label_map[predicted_label]

num_correct = 0

# Evaluate the Q&A performance
for example in test_dataset:
    question = example['question']
    # Include the context field in the prediction
    context = example['context']
    # Extract true answer from 'final_decision' column
    true_answer = example['final_decision']
    # Use model to predict the answer
    predicted_answer = prediction_answer(question, context)

    print(f"Question: {question}")
    print(f"Context: {context}")
    print(f"True Answer: {true_answer}")
    print(f"Predicted Answer: {predicted_answer}")
    print("="*80)

    # Calculate accuracy
    if true_answer == predicted_answer:
        num_correct += 1

# Print accuracy
accuracy = num_correct / len(test_dataset)
print(f"Accuracy: {accuracy}")